In [217]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec as w2v
import math
from nltk import ngrams

In [5]:
newsgroup_df = pd.read_csv("20newsgroup_preprocessed.csv", sep=';')
newsgroup_df.head()

,target,text,text_cleaned
0,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: A...,atheist resources addresses atheist organizati...
1,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: A...,begin pgp signed message introduction atheism ...
2,alt.atheism,From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...,article charley wingate writes well john quite...
3,alt.atheism,From: mathew <mathew@mantis.co.uk>\nSubject: R...,kings become philosophers philosophers become ...
4,alt.atheism,From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...,article bob mcgwier writes however hate econom...


In [141]:
D = newsgroup_df['text_cleaned']
D = D.dropna()
tokenizedD = D.str.split()

In [15]:
V = []
for d in D:
    words = d
    for word in words:
        if word not in V:
            V.append(word)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\tripa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tripa\AppData\Local\Temp\ipykernel_27796\1438830031.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\tripa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\tripa\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\tripa\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1012, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c:\Users\tripa\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 865, in structured_traceb

In [17]:
tfidf = TfidfVectorizer()
tfidf

TfidfVectorizer()

In [35]:
X = tfidf.fit_transform(D[:9000]).toarray()
X.shape

(9000, 101185)

In [181]:
lsa = TruncatedSVD(n_components=100)

In [182]:
V_t = lsa.fit_transform(X)
V_t.shape

(9000, 100)

In [183]:
K = V_t.argmax(axis=1).tolist()
tfidf.get_feature_names_out()

array(['aa', 'aaa', 'aaaa', ..., 'ñaustin', 'úz', 'ÿhooked'], dtype=object)

In [184]:
def get_top_n_words(n):
    
    top_indices = []
    # Get top words for every topic we need
    for topic in range(100):
        temp_vector = 0
        for i in range(len(K)):
            # If the key is same as topic number, we add the key position to the vector of the topic
            if K[i] == topic:
                temp_vector += X[i]
        temp_vector_array = temp_vector
        # For each topic we get the n top words by sorting the vector
        # print(temp_vector_array.shape)
        # print(np.flip(np.argsort(temp_vector_array))[:n], 0)
        # print(tfidf.get_feature_names_out()[94753])
        top_n_word_indices = np.flip(np.argsort(temp_vector_array),0)[:n]
        top_indices.append(top_n_word_indices)
      
    words = []
    for topic in top_indices:
        topic_words = []
        for index in topic:
            word = tfidf.get_feature_names_out()[index]
            topic_words.append(word)
        words.append(" ".join(topic_words))
       
    return words

In [185]:
top_n_words = get_top_n_words(8)

for i in range(len(top_n_words)):
    print(f"Topic {i+1}: ", top_n_words[i])

Topic 1:  would writes one like article know get thanks
Topic 2:  drive mb card controller disk meg floppy scsi
Topic 3:  scsi drive ide drives controller hard mb disk
Topic 4:  windows dos os nt svein apps ini emm
Topic 5:  card video dx bus mhz drivers vlb ati
Topic 6:  god atheism faith religion believe atheists exist atheist
Topic 7:  window bit image bits display images visual colormap
Topic 8:  window monitor expose event manager problem colors screen
Topic 9:  sale condition offer excellent shipping asking price email
Topic 10:  window card font windows irq tonis cornerstone com
Topic 11:  mhz dx thanks clock advance speed window motherboard
Topic 12:  car cars dealer tax ford insurance new safety
Topic 13:  com port modem irq serial ports printer mouse
Topic 14:  com irq interrupt card bus port devices ports
Topic 15:  file swap ns macintosh sipps sale please meg
Topic 16:  file manager print annese soundexe drag article doug
Topic 17:  team year games cubs game pitching braves

In [186]:
U = lsa.components_
U.shape


(100, 101185)

In [187]:
U = U.transpose()
U.shape

(101185, 100)

In [188]:
w = w2v(sentences=tokenizedD, window=10)

In [189]:
w.wv.most_similar('asia')

[('islands', 0.8994759321212769),
 ('spain', 0.8929166197776794),
 ('athletic', 0.8906872868537903),
 ('arabian', 0.8889362812042236),
 ('corfu', 0.8880494832992554),
 ('qatar', 0.887898325920105),
 ('persecutions', 0.8873692750930786),
 ('sultan', 0.8865421414375305),
 ('republics', 0.8860727548599243),
 ('alliance', 0.8832847476005554)]

In [210]:
np.random.seed(seed=101)
V = []
for i in range(len(U)):
    V.append(np.random.random(100)/10)

V = np.array(V)


In [212]:
Theta = np.concatenate((V,U))
Theta.shape

(202370, 100)

In [236]:
def sigmoid(x):
    return 1/(1+math.exp(-x))


0.565861831899503

In [234]:
n = 1
unigram = np.array(list(ngrams(tfidf.get_feature_names_out(), n)))
unigram = unigram.reshape((101185,))

def sample():
    return np.random.choice(unigram, 10)

sample()

array(['batting', 'justifications', 'ikqsilmct', 'vsqge', 'hypotheses',
       'fgrgwmwrgwmwahf', 'nkjfjrlbktndmnfxr', 'gditho', 'lkg',
       'galactus'], dtype='<U171')

In [237]:
def J_t(u0: np.ndarray, vC: np.ndarray):
    a = math.log(sigmoid(np.dot(u0,vC)))
    b = 0
    negative_sample = sample()
    for j in range(len(negative_sample)):
        b += math.log(sigmoid(-np.dot(negative_sample[j], vC)))